In [1]:
from get_html import get_html
from get_html import get_result
from setting import json_head,proj_dir
from page import get_index
from page import page
from allcsv import allcsv
from item import item
from pathlib import Path
import pandas as pd
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
#from certifycode import crack
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By


In [2]:
# cities = ['hrb','dq','qqhr','mdj','suihua','jms','jixi','sys','hegang','heihe','yich','qth','dxal','shanda','shzhaodong','zhaozhou']
cities = ['qth','dxal','shanda','shzhaodong','zhaozhou']
for city in cities:
#leixing = 厂房、仓库、土地
    for leixing in ["tudi","changfang","cangkucf"]: 
        pn_url=f"https://{city}.58.com/{leixing}/" # 网址
        print(f'开始{city}的{leixing}')
        if __name__ == "__main__":
            '''pathlib.Path.mkdir
            mkdir(parents=True, exist_ok=True)
            parents：如果父目录不存在，是否创建父目录。
            exist_ok：只有在目录不存在时创建目录，目录已存在时不会抛出异常。
            '''
            Path(f"{proj_dir}/{city}/{leixing}/website").mkdir(parents=True, exist_ok=True)# 创建文件夹
            index=get_index(city,leixing,pn_url) # 获取最大页码-找网页规律
            print(f'页数={index}')

            if index!=0:
                startpage=1
                endpage=index # 最大页码
                page(startpage,endpage,pn_url=pn_url,city=city,leixing=leixing) # 获取每一页的网址，有异常则打印页码
                # page(startpage=4,endpage=8,pn_url=pn_url,city=city,leixing=leixing)    
            html_cnt = allcsv(city,leixing) # 每个文件夹下的html中的网址写入csv,返回pn_html个数

            startrow=0
            if html_cnt>0:
                item(startrow, city=city,leixing=leixing) # csv中的每个网址内容get并解析到website文件夹中


开始qthtudi
index=0
qthtudi搜索暂无结果
开始qthchangfang
index=2


100%|██████████| 36/36 [02:39<00:00,  4.44s/it]


开始qthcangkucf
index=2


100%|██████████| 12/12 [00:52<00:00,  4.41s/it]


开始dxaltudi
index=0
dxaltudi搜索暂无结果
开始dxalchangfang
index=0
dxalchangfang搜索暂无结果
开始dxalcangkucf
index=0
dxalcangkucf搜索暂无结果
开始shandatudi
index=2


...

开始shzhaodongchangfang
index=2


100%|██████████| 23/23 [01:41<00:00,  4.43s/it]


开始shzhaodongcangkucf
index=2


100%|██████████| 9/9 [00:40<00:00,  4.49s/it]


开始zhaozhoutudi
index=0
zhaozhoutudi搜索暂无结果
开始zhaozhouchangfang
index=0
zhaozhouchangfang搜索暂无结果
开始zhaozhoucangkucf
index=0
zhaozhoucangkucf搜索暂无结果


In [17]:
from setting import proj_dir
import glob
from bs4 import BeautifulSoup as bf
import pandas as pd
# 每个文件夹下的html中的网址写入csv

def allcsv(city,leixing):
    html_list = glob.glob(f"{proj_dir}/{city}/{leixing}/*.html")  # 查看同文件夹下的html文件列表
    if len(html_list)>0:
        f = open(f"{proj_dir}/{city}/{leixing}/{leixing}.csv", mode="a")
        for result in html_list:  # 循环读取同文件夹下的html文件
            obj = bf(open(result, encoding="utf-8"), features="html.parser")  # features值可为lxml
            for item in obj.select('ul[class="list-main-style"] a'):
                detail_url = item.get("href")
                if 'https:' not in str(detail_url):
                    detail_url = f"https:{detail_url}"
                    f.write("%s\n" % (detail_url))
                else:
                    f.write("%s\n" % (detail_url))
        f.close()
        data = pd.read_csv(f"{proj_dir}/{city}/{leixing}/{leixing}.csv", header=None)
        data = data.iloc[:, 0].astype("str").str.split("prd", expand=True)[0] # expand让分割内容变成2列
        data = data.drop_duplicates() # 去重
    else:
        print(f'{city}搜索暂无结果')
allcsv(city,leixing)

qth搜索暂无结果


In [14]:
data = pd.read_csv(f"{proj_dir}/{city}/{leixing}/{leixing}.csv",header=None)
data = data.iloc[:, 0].astype("str")
endrow=data.shape[0]
for i in tqdm(range(startrow, endrow)):
    item_url = data.iloc[i]

EmptyDataError: No columns to parse from file

In [6]:
city,leixing

('qth', 'tudi')

In [4]:

kw = "pagelist" # 关键词
reponse = get_html(pn_url, kw, json_head) # request.get返回网址响应(添加请求头和参数)
obj = get_result(reponse) # bs4解析html
if '搜索暂无结果，查看下其他内容吧' not in obj.text:
    if obj.find("div", class_="pager").find_all("a"):
        page = obj.find("div", class_="pager").find_all("a") # 定位页码标签
        if len(page)>3: # 网址>2页
            maxpage = page[2].get("href") # 例如https://qqhr.58.com/changfang/pn2/ 齐齐哈尔厂房第2页
            new_url = maxpage[: maxpage.index("?") + 1].split("pn")
            index = int(new_url[1].split("/")[0]) + 1 
        else:
            maxpage = page[-2].get("href") # 倒数第二个（最后1个是"下一页"）
            new_url = maxpage[: maxpage.index("?") + 1].split("pn")
            index = int(new_url[1].split("/")[0]) + 1 
    elif obj.find("div", class_="pager"): # 网页源码找不到class_="pager"
        index = 2 
    else: # ???
        index=0
else: # 'NoneType' object has no attribute 'find_all'
    index=0
index

0

In [6]:
obj.text

'\n\n\n\n【七台河土地出租|七台河土地租赁网，七台河土地招租价格信息】-七台河58同城\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n免费发布\n\n\n\n\n\n\n\n\n\n\n\n\n                            桃南街星光巷\n                        \n\n\n\n\n\n\n\n\n租房\n\n\n二手房\n\n\n商铺\n\n\n生意转让\n\n\n写字楼\n\n\n厂房\n\n\n仓库\n\n\n土地\n\n\n车位\n\n\n\n\n\n\n                七台河58同城\n            \n             >\n            \n                七台河房产信息\n            \n             >\n            \n                七台河土地出租\n            \n\n\n\n\n\n                        出租\n                    \n\n\n\n                        出售\n                    \n\n\n\n                        转让\n                    \n\n\n\n\n\n\n\n位置：\n区域\n地图找房\n\n\n\n\n\n\n全七台河\n桃山\n新兴\n茄子河\n勃利\n七台河周边\n\n\n\n\n\n\n\n\n\n\n\n面积：\n\n不限\n100m²以下\n100-500m²\n500-1000m²\n1000-2000m²\n2000-5000m²\n5000-8000m²\n8000-12000m²\n12000m²以上\n\n\n - \n\nm²\n\n\n\n\n\n\n\n\n\n 租金：\n\n\n不限\n2元/m²/天以下\n2-3元/m²/天\n3-5元/m²/天\n5-6元/m²/天\n6-8元/m²/天\n8-10元/m²/天\n10元/m²/天以上\n\n\n\n                    不限\n      

In [21]:
# 单个城市测试
city='jms'
for leixing in ["cangkucf","changfang","tudi"]: 
    pn_url=f"https://{city}.58.com/{leixing}/" # 网址

    if __name__ == "__main__":
        '''pathlib.Path.mkdir
        mkdir(parents=True, exist_ok=True)
        parents：如果父目录不存在，是否创建父目录。
        exist_ok：只有在目录不存在时创建目录，目录已存在时不会抛出异常。
        '''
        Path(f"{proj_dir}/{city}/{leixing}/website").mkdir(parents=True, exist_ok=True)# 创建文件夹
        index=get_index(city,leixing,pn_url) # 获取最大页码-找网页规律
        print(f'index={index}')

        if index!=0:
            startpage=1
            endpage=index # 最大页码
            page(startpage,endpage,pn_url=pn_url,city=city,leixing=leixing) # 获取每一页的网址，有异常则打印页码
            # page(startpage=4,endpage=8,pn_url=pn_url,city=city,leixing=leixing)    
        allcsv(city,leixing) # 每个文件夹下的html中的网址写入csv

        startrow=0
        if startrow==0:
            item(startrow, city=city,leixing=leixing) # csv中的每个网址内容get并解析到website文件夹中

index=3


100%|██████████| 43/43 [03:12<00:00,  4.47s/it]


index=3


100%|██████████| 43/43 [03:09<00:00,  4.41s/it]


index=2


100%|██████████| 8/8 [00:35<00:00,  4.42s/it]


In [2]:
pn_url='https://dl.58.com/tudi/'

In [3]:
from get_html import get_html,get_result
from setting import json_head
import time
from tqdm import tqdm
from setting import stop_word1,stop_word2,proj_dir,limit
# from certifycode import crack
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
kw = "pagelist" # 关键词
reponse = get_html(pn_url, kw, json_head) # request.get返回网址响应(添加请求头和参数)
obj = get_result(reponse) # bs4解析html
if obj.find("div", class_="pager").find_all("a"):
    page = obj.find("div", class_="pager").find_all("a") # 定位页码标签
    if len(page)>3:
        maxpage = page[2].get("href") # 例如https://qqhr.58.com/changfang/pn2/ 齐齐哈尔厂房第2页
        new_url = maxpage[: maxpage.index("?") + 1].split("pn")
        index = int(new_url[1].split("/")[0]) + 1
    else:
        maxpage = page[-2].get("href")
        new_url = maxpage[: maxpage.index("?") + 1].split("pn")
        index = int(new_url[1].split("/")[0]) + 1 
elif obj.find("div", class_="pager"):
    index = 2
else:
    index=0
index

5

In [11]:
maxpage


'https://dl.58.com/tudi/pn4/?wd=pagelist&gposLastIndex=35&skuInserted=0&FGTID=186520419219925209409196034&maxInserted=0'

In [19]:
maxpage[: maxpage.index("?")-1].split('/pn')[-1]

'4'

In [7]:
from setting import proj_dir
import glob
from bs4 import BeautifulSoup as bf
import pandas as pd


In [28]:
html_list = glob.glob(f"{proj_dir}/{city}/{leixing}/*.html")  # 查看同文件夹下的html文件列表
f = open(f"{proj_dir}/{city}/{leixing}/{leixing}.csv", mode="a")
for result in html_list:  # 循环读取同文件夹下的html文件
    obj = bf(open(result, encoding="utf-8"), features="html.parser")  # features值可为lxml
    for item in obj.select('ul[class="list-main-style"] a'):
        detail_url = item.get("href")
        if 'https:' not in str(detail_url):
            detail_url = f"https:{detail_url}"
            f.write("%s\n" % (detail_url))
        else:
            f.write("%s\n" % (detail_url))
        print(f'→{detail_url}')
f.close()
data = pd.read_csv(f"{proj_dir}/{city}/{leixing}/{leixing}.csv", header=None)
data = data.iloc[:, 0].astype("str").str.split("prd", expand=True)[0] # expand让分割内容变成2列
data = data.drop_duplicates() # 去重
data.to_csv(f"{proj_dir}/{city}/{leixing}/{leixing}.csv",mode='w',header=None,index=False)

→https://qqhr.58.com/fangchan/53460389113492x.shtml?prd=L9PR%2FN9l6x2ue7W6BPEOdAxuUYbQ98LiLMz0k3yqRde38BNcQ8uPyg8AsxiH661W&houseId=2920888122321929&gpos=1&legotopurl=%2F%2Flegoclick.58.com%2Fjump%3Ftarget%3DszqQ0y6zsWN3shPfUiqhmyOMmv66UBtdn1Evnjn3rHDQn1EOnM980v6YUykKuadBrHR6nHF-PzYQrH91sHEQuWTVmWR-PBdbn1mLrjFhPH93PhcKPWn1PWcYrHnQP1NkPH9KTHcOnWT3rj9QnWc1nWDOnWbKnTDQnjTKnikQP9DdrjN1THDvP103rHDOnHNvnH0KnEDdrad_pgPYXhEVOuyAOsBpOuzkOsJMTHDKnEDKsHDKTHDvn1T3Pj0YPW0LPWbvnH9Yn1NKP97kmgF6Ugn1PHmlnBQkmgF6UgnznjnlnEDQTHNYryRBPWI-syR-PjNVPAwBraYOujcOsyRbujmLnHbduHnLm9DQPWnkrjELPjmLrj9zrjNzn1NzTHDvn1T3Pj0YPW0LrHTdrjN3PH0KTEDKTEDVTEDKTgKiyZ01wdKCrZmzHDCLIvqNijFdEy0q5EDznjTQrhw6rjCOnjTLrWEkPjDlmyw6njC1uH--rWPbuHEluHwBnEDOn10LPjmzn10YnH0dnTDkTEDQsjDvTgVqTHDknTDQnjTzrHTkTyE3PHu-nWE1sHNQuH9VPj9OmzY3nj6WsH0znjcdmWNYm1-WPTDKnTDKTHTKPH9dnzkdrjNdsjnvPjbLTHDKUMR_UTDQryDQPA76mvRbrAwbuAE1&referinfo=false&utm_source=&spm=&positionType=legotophouse&lgtid_shangyetie=d856e243-51e8-489c-808c-72025b54c9c4&cocoTestTyp

In [11]:
detail_url

'https://qqhr.58.com/fangchan/53375765772553x.shtml?prd=L9PR%2FN9l6x2ue7W6BPEOdAxuUYbQ98LiLMz0k3yqRdevk3talB%2BZkKrc7WBZrhdE&houseId=2910056334531597&gpos=34&positionType=houserecommendlist&cocoTestType=coco_area_expand_test&legoCocoTestType=coco_area_expand_test&filterJson=e30&jx_abtest=ZWljX3N5ZGNfcGNfcmVjb21tZW5kX3Rlc3QsY29jb19hcmVhX2V4cGFuZF90ZXN0LHVzZXJfcGhhc2VfbA&list_type=main'

In [ ]:
#from tudiparse import parse_tudi
from fangchanparse import parse_fc
from setting import proj_dir
import pandas as pd

citylist=pd.read_excel(f'{proj_dir}/citylist.xlsx')
pro="黑龙江"
df=pd.read_excel(f'{proj_dir}/租金.xlsx')
for city in citylist.loc[citylist['pro']==pro,'index']:
    for leixing in ['tudi','changfang','cangkucf']:
        parse_fc(city,leixing,df)
df.to_excel(f'{proj_dir}/{pro}租金.xlsx',index=False)


